# Vocal tract modelling

In this notebook, the geometry of a vocal tract will be inferred using the boundary element method (BEM) and a genetic algorithm. First, the BEM will be used to build an acoustic model of a vocal tract. 

In [1]:
include("../BEM_base.jl"); # The BEM algorithms are defined in the BEM_base

In [2]:
# Set the msh file location
file = "../dados/vocal_tract/vocal_tract_A_bothclosed.msh";
# Build the domain points
L = 140; # Length of the vocal tract
n_pint = 50; # Number of domain points
PONTOS_int = zeros(n_pint,4);
delta = 1; # distance from both ends 
passo = (L-2*delta)/(n_pint-1);
for i = 1:n_pint
    PONTOS_int[i,:] = [i 0 0 delta+(i-1)*passo];
end
# Set the boundary conditions for each face. Vowel /A/ model has 30 faces
BCFace = ones(30,3);
BCFace[:,3] = 0;
BCFace[1,:] = [1 0 -1]; # Neumann (flux = 1) to the Glotis
BCFace[30,:] = [30 0 0]; # Dirichlet (pressure = 0) to the mouth
CW = 343*1000; # Speed of sound in mm/s
k = 774/CW/2/pi; # Set the wavenumber
# Solve the BEM model
u,q,uint,qint = BEM_base(file,PONTOS_int,BCFace,k, "wave");

Importing mesh...
  7.890013 seconds (717.68 k allocations: 37.792 MiB, 0.25% gc time)
Building G and H matrices...
 11.446589 seconds (224.64 M allocations: 10.573 GiB, 17.07% gc time)
Applying boundary conditions to build A and b for the linear system...
  0.279270 seconds (3.95 k allocations: 24.210 MiB, 0.75% gc time)
Solving the linear system...
  1.255593 seconds (401.61 k allocations: 30.392 MiB, 0.73% gc time)
Separating acoustic pressure from flux...
  0.048543 seconds (3.62 k allocations: 222.430 KiB)
Solving for domain points.
  0.672024 seconds (13.65 M allocations: 658.150 MiB, 16.65% gc time)
  1.518122 seconds (31.97 M allocations: 1.196 GiB, 12.89% gc time)


In [3]:
# Graph the results
#plot(PONTOS_int[:,4],real(uint),label=L"$\phi$ BEM")
plot(PONTOS_int[:,4],real(uint))
#plot(PONTOS_int[:,4],real(qint),label=L"$q$ BEM")
#grid(1)
#legend()

LoadError: [91mUndefVarError: plot not defined[39m

In [4]:
#plot(PONTOS_int[:,4],real(qint),label=L"$q$ BEM")
plot(PONTOS_int[:,4],real(qint))
#grid(1)
#legend()

LoadError: [91mUndefVarError: plot not defined[39m

In [5]:
@time mshinfo = const3D_tri.lermsh(file,3) #Read the mesh generated 

  1.040323 seconds (313.20 k allocations: 15.526 MiB)


([1.0 -7.5694 0.0 0.0; 2.0 0.0 0.0 0.0; … ; 426.0 7.87848 7.87848 140.0; 427.0 -7.87848 -7.87848 140.0], [1 46 … 222 1; 2 3 … 51 1; … ; 819 208 … 427 30; 820 213 … 426 30], Array{Any}(0,2), Any[15, 15, 15, 15, 15, 15, 15, 15, 15, 15  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
NOS_GEO,ELEM,elemint,CDC = mshinfo

([1.0 -7.5694 0.0 0.0; 2.0 0.0 0.0 0.0; … ; 426.0 7.87848 7.87848 140.0; 427.0 -7.87848 -7.87848 140.0], [1 46 … 222 1; 2 3 … 51 1; … ; 819 208 … 427 30; 820 213 … 426 30], Array{Any}(0,2), Any[15, 15, 15, 15, 15, 15, 15, 15, 15, 15  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
ELEM

820×5 Array{Int64,2}:
   1   46    1  222   1
   2    3  221   51   1
   3  220  226  223   1
   4  220  225  224   1
   5   52  224   53   1
   6   47  223   48   1
   7   54  225   55   1
   8   49  226   50   1
   9  221  220  224   1
  10  220  223  222   1
  11  221  226  220   1
  12  220  222  225   1
  13   53  231   54   1
   ⋮                   
 809   45  419  424  30
 810   43  418  425  30
 811  422  418  427  30
 812  420  419  426  30
 813  421  425  418  30
 814  423  424  419  30
 815   45  424  212  30
 816   43  425  217  30
 817  212  424  423  30
 818  217  425  421  30
 819  208  209  427  30
 820  213  214  426  30